<a href="https://colab.research.google.com/github/anthonytranhoang/Credit_card_fraud/blob/Sima_branch/Project_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Ensure skl2onnx is installed for model conversion to ONNX format.
!pip install skl2onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.4/298.4 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 92.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 58.0 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.17.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 3.20.2 which is incompatible.
tensorflow-metadata 1.16.1 requires protobuf<4.21,>=3.20.3; python_version < "3.11", but you have protobuf 3.20.2 which is incompatible.


In [5]:
# Import all the dependencies for ML project
import pandas as pd  # Data manipulation
import numpy as np  # Numerical operations
import matplotlib.pyplot as plt  # Plotting
import seaborn as sns  # Data visualization
import psycopg2  # PostgreSQL adapter
from psycopg2 import sql  # SQL queries
from sklearn.preprocessing import LabelEncoder, StandardScaler  # Preprocessing
from xgboost import XGBClassifier  # XGBoost classifier
from sklearn.model_selection import train_test_split  # Train-test split
from sklearn.metrics import classification_report, accuracy_score  # Metrics
from sklearn.ensemble import RandomForestClassifier  # Random Forest classifier
from sklearn.utils import resample  # Resampling utilities
from imblearn.over_sampling import SMOTE  # Handle imbalanced data
import joblib  # Model saving
from skl2onnx import convert_sklearn  # Convert to ONNX
from skl2onnx.common.data_types import FloatTensorType  # ONNX types
import onnx  # ONNX support
import pickle  # Object serialization

In [6]:
# Define the connection parameters
host = 'database-1.cxu0eg8y6y4k.ap-southeast-2.rds.amazonaws.com'
database = 'Finance_db'
user = 'postgres'
password = 'Radin5286'
port = '5432'

In [7]:
# Establish the connection
try:
    conn = psycopg2.connect(
        host=host,
        database=database,
        user=user,
        password=password,
        port=port
    )
    print("Connection successful!")
except Exception as e:
    print(f"Error: {e}")

Connection successful!


In [8]:
query = """
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema = 'public'
      AND table_type = 'BASE TABLE';
    """

    # Execute the query and load into a DataFrame
df_tables = pd.read_sql(query, conn)

    # Display the DataFrame containing table names
print(df_tables)


<ipython-input-8-9047a232e4f4>:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_tables = pd.read_sql(query, conn)


              table_name
0  classification_report
1                finance


In [9]:
query = "SELECT * FROM finance;"

In [10]:
finance_df = pd.read_sql(query, conn)
finance_df.head(2)

<ipython-input-10-f764c01b7782>:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  finance_df = pd.read_sql(query, conn)


,fraud_bool,income,name_email_similarity,prev_address_months_count,current_address_months_count,customer_age,days_since_request,intended_balcon_amount,payment_type,zip_count_4w,velocity_6h,velocity_24h,velocity_4w,bank_branch_count_8w,date_of_birth_distinct_emails_4w,employment_status,credit_risk_score,email_is_free,housing_status,phone_home_valid,phone_mobile_valid,bank_months_count,has_other_cards,proposed_credit_limit,foreign_request,source,session_length_in_minutes,device_os,keep_alive_session,device_distinct_emails_8w,device_fraud_count,month
0,0,0.3,0.986506,-1,25,40,0.006735,102.453711,AA,1059,13096.035018,7850.955007,6742.080561,5,5,CB,163,1,BC,0,1,9,0,1500.0,0,INTERNET,16.224843,linux,1,1,0,0
1,0,0.8,0.617426,-1,89,20,0.010095,-0.849551,AD,1658,9223.283431,5745.251481,5941.664859,3,18,CA,154,1,BC,1,1,2,0,1500.0,0,INTERNET,3.363854,other,1,1,0,0


In [11]:
conn.close()

In [12]:
finance_df.dtypes
#Checking dtypes.

,0
fraud_bool,int64
income,float64
name_email_similarity,float64
prev_address_months_count,int64
current_address_months_count,int64
customer_age,int64
days_since_request,float64
intended_balcon_amount,float64
payment_type,object
zip_count_4w,int64


In [13]:
finance_df['fraud_bool'].unique()

array([0, 1])

In [14]:
finance_df['fraud_bool'].value_counts()

,count
fraud_bool,
0,988971
1,11029


In [15]:
finance_df.columns

Index(['fraud_bool', 'income', 'name_email_similarity',
       'prev_address_months_count', 'current_address_months_count',
       'customer_age', 'days_since_request', 'intended_balcon_amount',
       'payment_type', 'zip_count_4w', 'velocity_6h', 'velocity_24h',
       'velocity_4w', 'bank_branch_count_8w',
       'date_of_birth_distinct_emails_4w', 'employment_status',
       'credit_risk_score', 'email_is_free', 'housing_status',
       'phone_home_valid', 'phone_mobile_valid', 'bank_months_count',
       'has_other_cards', 'proposed_credit_limit', 'foreign_request', 'source',
       'session_length_in_minutes', 'device_os', 'keep_alive_session',
       'device_distinct_emails_8w', 'device_fraud_count', 'month'],
      dtype='object')

In [17]:

 finance_df[['fraud_bool', 'income', 'name_email_similarity',
       'prev_address_months_count', 'current_address_months_count',
        'customer_age', 'days_since_request', 'intended_balcon_amount', 'zip_count_4w', 'velocity_6h', 'velocity_24h',
       'velocity_4w', 'bank_branch_count_8w',
        'date_of_birth_distinct_emails_4w',
       'credit_risk_score', 'email_is_free',
        'phone_home_valid', 'phone_mobile_valid', 'bank_months_count',
        'has_other_cards', 'proposed_credit_limit', 'foreign_request',
        'session_length_in_minutes', 'keep_alive_session',
        'device_distinct_emails_8w', 'device_fraud_count', 'month']].corr()
# Looking at the correlation to see which features are more important.



,fraud_bool,income,name_email_similarity,prev_address_months_count,current_address_months_count,customer_age,days_since_request,intended_balcon_amount,zip_count_4w,velocity_6h,velocity_24h,velocity_4w,bank_branch_count_8w,date_of_birth_distinct_emails_4w,credit_risk_score,email_is_free,phone_home_valid,phone_mobile_valid,bank_months_count,has_other_cards,proposed_credit_limit,foreign_request,session_length_in_minutes,keep_alive_session,device_distinct_emails_8w,device_fraud_count,month
fraud_bool,1.000000,0.045079,-0.036720,-0.026031,0.033701,0.062959,0.000567,-0.024524,0.005212,-0.016892,-0.011183,-0.011536,-0.011577,-0.043224,0.070624,0.027758,-0.035128,-0.013180,-0.003222,-0.035156,0.068907,0.016885,0.008999,-0.050296,0.035704,NaN,0.013250
income,0.045079,1.000000,-0.038519,0.018209,-0.036985,0.125822,-0.015877,0.059046,-0.080581,-0.096431,-0.101269,-0.106333,0.007474,-0.062348,0.171447,-0.024900,0.024938,0.014979,-0.004514,0.086861,0.108868,0.011452,-0.055430,-0.043584,-0.009992,NaN,0.116479
name_email_similarity,-0.036720,-0.038519,1.000000,-0.018186,0.050305,-0.048902,-0.007736,0.029040,0.025114,0.024881,0.035221,0.039968,-0.001470,0.022994,0.049126,-0.066897,-0.002931,0.008966,-0.006351,0.011021,0.073159,-0.015224,0.005998,0.014342,-0.017804,NaN,-0.042949
prev_address_months_count,-0.026031,0.018209,-0.018186,1.000000,-0.271730,-0.069178,0.078107,-0.009428,-0.029638,-0.001265,0.009020,0.005413,-0.040628,0.059560,-0.016104,-0.014348,-0.053353,0.004757,-0.045550,-0.042949,-0.028474,0.019242,0.051425,0.031672,0.011924,NaN,-0.001793
current_address_months_count,0.033701,-0.036985,0.050305,-0.271730,1.000000,0.142863,-0.057836,0.057601,0.047286,0.027508,0.018753,0.026172,0.054202,-0.174714,0.094976,-0.079519,0.129102,-0.088268,0.071997,0.060732,0.130594,-0.012355,-0.033574,-0.050539,-0.000760,NaN,-0.024541
customer_age,0.062959,0.125822,-0.048902,-0.069178,0.142863,1.000000,-0.035999,-0.013000,-0.014176,-0.022063,-0.010030,-0.004303,0.048412,-0.420173,0.166094,-0.000357,0.212713,-0.147098,0.012907,0.108743,0.148857,-0.001606,0.025049,-0.024272,0.025709,NaN,0.005694
days_since_request,0.000567,-0.015877,-0.007736,0.078107,-0.057836,-0.035999,1.000000,0.019782,0.000249,0.027968,0.023869,0.017706,-0.018173,0.018731,-0.088141,0.008163,-0.042009,0.005130,-0.027460,-0.048861,-0.068670,0.001066,0.049613,0.000337,0.009025,NaN,-0.017938
intended_balcon_amount,-0.024524,0.059046,0.029040,-0.009428,0.057601,-0.013000,0.019782,1.000000,0.002251,0.042849,0.064080,0.051407,0.102919,-0.027343,0.002058,-0.014754,0.000019,0.038625,0.178374,0.096676,0.057860,0.007933,0.022811,-0.020689,-0.019910,NaN,-0.043070
zip_count_4w,0.005212,-0.080581,0.025114,-0.029638,0.047286,-0.014176,0.000249,0.002251,1.000000,0.142372,0.201186,0.302955,0.005958,0.126885,-0.102645,0.021731,-0.035661,0.012122,0.053637,-0.015015,-0.011651,0.017212,0.039687,0.022283,0.020550,NaN,-0.287203
velocity_6h,-0.016892,-0.096431,0.024881,-0.001265,0.027508,-0.022063,0.027968,0.042849,0.142372,1.000000,0.464003,0.400254,0.020140,0.118752,-0.145254,0.028379,-0.036820,-0.010751,0.014874,-0.007049,-0.025005,-0.000450,0.054491,0.002797,0.030030,NaN,-0.409148


In [18]:
# looking for the null values in a data.
finance_df.isnull().sum()

,0
fraud_bool,0
income,0
name_email_similarity,0
prev_address_months_count,0
current_address_months_count,0
customer_age,0
days_since_request,0
intended_balcon_amount,0
payment_type,0
zip_count_4w,0


In [19]:
# saving the dataframe to csv in local storage.
finance_df.to_csv('finance.csv')

In [20]:
finance_df.head(10)

,fraud_bool,income,name_email_similarity,prev_address_months_count,current_address_months_count,customer_age,days_since_request,intended_balcon_amount,payment_type,zip_count_4w,velocity_6h,velocity_24h,velocity_4w,bank_branch_count_8w,date_of_birth_distinct_emails_4w,employment_status,credit_risk_score,email_is_free,housing_status,phone_home_valid,phone_mobile_valid,bank_months_count,has_other_cards,proposed_credit_limit,foreign_request,source,session_length_in_minutes,device_os,keep_alive_session,device_distinct_emails_8w,device_fraud_count,month
0,0,0.3,0.986506,-1,25,40,0.006735,102.453711,AA,1059,13096.035018,7850.955007,6742.080561,5,5,CB,163,1,BC,0,1,9,0,1500.0,0,INTERNET,16.224843,linux,1,1,0,0
1,0,0.8,0.617426,-1,89,20,0.010095,-0.849551,AD,1658,9223.283431,5745.251481,5941.664859,3,18,CA,154,1,BC,1,1,2,0,1500.0,0,INTERNET,3.363854,other,1,1,0,0
2,0,0.8,0.996707,9,14,40,0.012316,-1.490386,AB,1095,4471.472149,5471.988958,5992.555113,15,11,CA,89,1,BC,0,1,30,0,200.0,0,INTERNET,22.730559,windows,0,1,0,0
3,0,0.6,0.475100,11,14,30,0.006991,-1.863101,AB,3483,14431.993621,6755.344479,5970.336831,11,13,CA,90,1,BC,0,1,1,0,200.0,0,INTERNET,15.215816,linux,1,1,0,0
4,0,0.9,0.842307,-1,29,40,5.742626,47.152498,AA,2339,7601.511579,5124.046930,5940.734212,1,6,CA,91,0,BC,1,1,26,0,200.0,0,INTERNET,3.743048,other,0,1,0,0
5,0,0.6,0.294840,-1,369,30,0.024232,-1.232556,AD,1204,11556.955514,7506.951276,6482.924037,705,5,CB,134,1,BE,1,1,30,0,200.0,0,INTERNET,6.987316,linux,1,1,0,0
6,0,0.2,0.773085,22,4,40,0.006919,-0.544676,AB,1998,11723.993606,7864.277144,6338.799156,28,8,CA,72,1,BC,1,1,1,0,200.0,0,INTERNET,28.199923,x11,1,1,0,0
7,0,0.8,0.153880,-1,103,40,0.045122,-1.101184,AB,1548,4999.555801,4526.861667,6426.790817,6,7,CA,163,0,BE,1,1,25,1,200.0,0,INTERNET,11.234264,other,1,1,0,0
8,0,0.3,0.523655,21,2,30,0.035206,-0.955737,AB,1781,6979.994002,4335.685346,6624.957942,2,10,CA,35,0,BC,1,0,2,0,200.0,0,INTERNET,5.329387,other,1,1,0,0
9,0,0.8,0.834475,-1,134,20,0.017245,-1.356393,AD,3113,7549.992086,6273.922110,6312.998835,14,20,CA,201,1,BD,1,1,15,0,1500.0,0,INTERNET,4.103970,other,1,1,0,0


In [21]:
finance_df['payment_type'].unique()

array(['AA', 'AD', 'AB', 'AC', 'AE'], dtype=object)

In [22]:
finance_df['employment_status'].unique()

array(['CB', 'CA', 'CC', 'CF', 'CD', 'CE', 'CG'], dtype=object)

In [23]:
finance_df['housing_status'].unique()

array(['BC', 'BE', 'BD', 'BA', 'BB', 'BF', 'BG'], dtype=object)

In [24]:
finance_df['source'].unique()

array(['INTERNET', 'TELEAPP'], dtype=object)

In [25]:
finance_df['device_os'].unique()

array(['linux', 'other', 'windows', 'x11', 'macintosh'], dtype=object)

In [26]:
# One-hot encoding 'employment_status'
finance_df1 = pd.get_dummies(finance_df, columns=['employment_status','payment_type','housing_status','device_os'], drop_first=True)
# drop_first=True prevents multicollinearity by dropping one category


In [27]:
finance_df1.head()

,fraud_bool,income,name_email_similarity,prev_address_months_count,current_address_months_count,customer_age,days_since_request,intended_balcon_amount,zip_count_4w,velocity_6h,velocity_24h,velocity_4w,bank_branch_count_8w,date_of_birth_distinct_emails_4w,credit_risk_score,email_is_free,phone_home_valid,phone_mobile_valid,bank_months_count,has_other_cards,proposed_credit_limit,foreign_request,source,session_length_in_minutes,keep_alive_session,device_distinct_emails_8w,device_fraud_count,month,employment_status_CB,employment_status_CC,employment_status_CD,employment_status_CE,employment_status_CF,employment_status_CG,payment_type_AB,payment_type_AC,payment_type_AD,payment_type_AE,housing_status_BB,housing_status_BC,housing_status_BD,housing_status_BE,housing_status_BF,housing_status_BG,device_os_macintosh,device_os_other,device_os_windows,device_os_x11
0,0,0.3,0.986506,-1,25,40,0.006735,102.453711,1059,13096.035018,7850.955007,6742.080561,5,5,163,1,0,1,9,0,1500.0,0,INTERNET,16.224843,1,1,0,0,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
1,0,0.8,0.617426,-1,89,20,0.010095,-0.849551,1658,9223.283431,5745.251481,5941.664859,3,18,154,1,1,1,2,0,1500.0,0,INTERNET,3.363854,1,1,0,0,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,True,False,False
2,0,0.8,0.996707,9,14,40,0.012316,-1.490386,1095,4471.472149,5471.988958,5992.555113,15,11,89,1,0,1,30,0,200.0,0,INTERNET,22.730559,0,1,0,0,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,True,False
3,0,0.6,0.475100,11,14,30,0.006991,-1.863101,3483,14431.993621,6755.344479,5970.336831,11,13,90,1,0,1,1,0,200.0,0,INTERNET,15.215816,1,1,0,0,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False
4,0,0.9,0.842307,-1,29,40,5.742626,47.152498,2339,7601.511579,5124.046930,5940.734212,1,6,91,0,1,1,26,0,200.0,0,INTERNET,3.743048,0,1,0,0,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False


In [28]:
# Example: Label encoding the 'payment_type' column
label_encoder = LabelEncoder()
finance_df1['source'] = label_encoder.fit_transform(finance_df1['source'])


In [29]:
finance_df1.dtypes

,0
fraud_bool,int64
income,float64
name_email_similarity,float64
prev_address_months_count,int64
current_address_months_count,int64
customer_age,int64
days_since_request,float64
intended_balcon_amount,float64
zip_count_4w,int64
velocity_6h,float64


In [30]:
# Separating the target variable and features
X = finance_df1.drop(columns=['fraud_bool'])
y = finance_df1['fraud_bool']

# Splitting the data into training and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initializing the XGBoost Classifier
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# Training the model
xgb_model.fit(X_train, y_train)

# Making predictions on the test set
y_pred = xgb_model.predict(X_test)

# Evaluating the model
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

accuracy, classification_rep


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [08:38:32] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


(0.98937,
 '              precision    recall  f1-score   support\n\n           0       0.99      1.00      0.99    197891\n           1       0.46      0.05      0.09      2109\n\n    accuracy                           0.99    200000\n   macro avg       0.73      0.52      0.54    200000\nweighted avg       0.98      0.99      0.99    200000\n')

In [31]:
print(f"\n{'Accuracy:':<15}{accuracy:.2f}\n")

# Pretty Print Classification Report
print("\nClassification Report:\n")
print(classification_rep)


Accuracy:      0.99


Classification Report:

              precision    recall  f1-score   support

           0       0.99      1.00      0.99    197891
           1       0.46      0.05      0.09      2109

    accuracy                           0.99    200000
   macro avg       0.73      0.52      0.54    200000
weighted avg       0.98      0.99      0.99    200000



### The XGBoost module was very accurate but it was not great at predicting the 1 class as there is a class imbalance. So we will try to oversample the class 1 and try different model to see it's precision.

first try


In [32]:
# Count the number of instances for each class
class_counts = finance_df1['fraud_bool'].value_counts()

# Determine the number of instances for the minority class (1s)
num_minority = class_counts.min()

# Separate majority and minority classes
df_majority = finance_df1[finance_df1['fraud_bool'] == 0]
df_minority = finance_df1[finance_df1['fraud_bool'] == 1]

# Randomly undersample the majority class
df_majority_undersampled = resample(df_majority,
                                     replace=False,  # do not want to replace
                                     n_samples=num_minority,  # match minority class
                                     random_state=42)  # reproducible results

# Combine undersampled majority class with original minority class
df_balanced = pd.concat([df_majority_undersampled, df_minority])

# Shuffle the new DataFrame
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# Separate features and target variable
X = df_balanced.drop('fraud_bool', axis=1)  # Features
y = df_balanced['fraud_bool']  # Target variable

# One-hot encode categorical variables
X = pd.get_dummies(X, drop_first=True)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train the model with balanced class weights
model = RandomForestClassifier(class_weight='balanced', random_state=42)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Print classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.81      0.80      3335
           1       0.80      0.78      0.79      3283

    accuracy                           0.80      6618
   macro avg       0.80      0.80      0.80      6618
weighted avg       0.80      0.80      0.80      6618



In [34]:
print(pd.Series(y_train).value_counts())

fraud_bool
1    692280
0    692279
Name: count, dtype: int64


second try ( The model used for integrate with Flask for the frontend UI )

In [33]:
# Select features and target
features = ['income', 'intended_balcon_amount', 'velocity_6h', 'velocity_24h', 'customer_age']
X = finance_df1[features]
y = finance_df1['fraud_bool']

# Apply SMOTE to balance the classes
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Scale the resampled data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_resampled)

# Train/Test Split (with stratification)
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_resampled, test_size=0.3, random_state=42, stratify=y_resampled
)

# Train the model
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred, target_names=["Non-Fraud", "Fraud"]))

KeyboardInterrupt: 

Saving the seccong try model to the ONNX file





In [ ]:
onnx_filename = 'model.onnx'
input_dim = X_train.shape[1]

try:
    onnx_model = convert_sklearn(model, initial_types=[('input', FloatTensorType([None, input_dim]))], target_opset=13)
    with open(onnx_filename, 'wb') as file:
        file.write(onnx_model.SerializeToString())
    print(f"Model saved to {onnx_filename}")
except Exception as e:
    print(f"Error converting or saving model: {e}")

Model saved to model.onnx


In [ ]:
# Create a StandardScaler object
scaler = StandardScaler()

# Fit the scaler to your data
scaler.fit(X)

# Save the fitted scaler to a file using pickle
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

print("Scaler saved to scaler.pkl")

Scaler saved to scaler.pkl


third try

In [ ]:
# Select features to train on
features = ['income', 'intended_balcon_amount', 'velocity_6h', 'velocity_24h', 'customer_age']


X = finance_df1[features]

# Instantiate and fit a new scaler with all the features
scaler = StandardScaler()
scaler.fit(X)

# Save the new scaler using pickle for consistency in your Flask app
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

# initial type for onnx conversion
initial_type = [('input', FloatTensorType([None, len(features)]))]


In [ ]:
# Select features to use in anomaly detection
features = ['intended_balcon_amount', 'velocity_6h', 'velocity_24h', 'customer_age']

# Scale the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(finance_df[features])

# Select features and target
X = finance_df1[features]
y = finance_df1['fraud_bool']

# Apply SMOTE to balance the classes
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42)

# Train the model with class weight
model = RandomForestClassifier(class_weight='balanced', random_state=42)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.97      0.98      0.98    296961
           1       0.98      0.97      0.98    296422

    accuracy                           0.98    593383
   macro avg       0.98      0.98      0.98    593383
weighted avg       0.98      0.98      0.98    593383

